<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

In [2]:
data = pd.read_csv('insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


**Согласно документации к данным:**

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [4]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

**Размеры обучающей выборки:**

In [5]:
print(features.shape)

(5000, 4)


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? 

**Ответ:** Не изменится. 

**Обоснование:** Пусть $P$ – обратимая матрица $<=> PP^{-1} = P^{-1}P = E$. 

Тогда 
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$

$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y = 
\\ = ((X^T X)P)^{-1} (P^T)^{-1} P^T X^T y = 
\\ = ((X^T X)P)^{-1} X^T y =
\\ = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

**Вывод**

После умножения матрицы признаков на обратимую матрицу $P$ и проведя несколько пребразований в формуле обучения, было доказано, что такое изменение признаков не влияет на результат $y$.

## Алгоритм преобразования

**Алгоритм**

1. Создание модели линейной регрессии

2. Обучение модели на исходных признаках $X$

3. Расчет метрики $R_2$ для предсказанных ответов

4. Генерация случайной квадратной матрицы $P_{nxn}$, где n – ширина матрицы $X$ (т.е. количество признаков)

5. Доказательство обратимости матрицы $P$

6. Преобразование исходных признаков $X$, путём домножения на матрицу $P$

7. Повтор шага 2. - 3. для матрицы $XP$

8. Сравнение полученнных метрик $R_2$ 

**Обоснование**


Для преобразования признаков сгенерируем квадратную матрицу $P$ размером $(nxn)$.
Необходимо доказать, что полученная матрица обратима:
$P$ – обратная матрица $<=>$ 
- $PP^{-1} = P^{-1}P = E$
- $det(P) \neq 0$.

Получим преобразованную матрицу: $X(m, n) \times P(n, n)$

После обучения модели *линейной регрессии* на исходных данных и на преобразованных, рассчитаем метрики $R_2$ и сравним. Разница между полученными ответами должна стремиться к нулю, и следовательно изменение признаков не повлияет на предсказание модели.

## Проверка алгоритма

- Обучние модели Линейной регрессии на исходных признаках

In [6]:
model = LinearRegression()

model.fit(features, target)
predictions_X = model.predict(features)
score_x = r2_score(target, predictions_X)
print('R2 для исходных признаков:', score_x)

R2 для исходных признаков: 0.42494550286668


- Генерация случайной квадратной матрицы $P_{nxn}$

In [7]:
n = features.shape[1]
def generate_invertible_matrix(size):
    try:
        matrix = np.random.normal(size=(size, size))
        np.linalg.inv(matrix)
    except np.linalg.LinAlgError:
        matrix = generate_invertible_matrix()
    
    return matrix
P = generate_invertible_matrix(n)

Мы сгенерировали квадратную обратимую матрицу $P$, теперь домножим её на матрицу признаков $X$ и повторим шаги 2-3 алгоритма:

In [8]:
model.fit(features @ P, target)
predictions_XP = model.predict(features @ P)
score_xp = r2_score(target, predictions_XP)
print('R2 для исходных признаков:', score_xp)

R2 для исходных признаков: 0.4249455028666743


**Вывод**

Был разработан метод преобразования данных, с помощью которого сложно восстановить персональную информацию. Метод заключается в том, чтобы преобразовать данные путем домножения на матрицу признаков случайно сгенерированную обратимую матрицу. По свойству обратимых квадратных матриц, данные изменения матрицы признаков не влияют на качество предсказания модели, что мы проверили с помощью применения описанного выше алгоритма:


In [9]:
print(score_x - score_xp)

5.662137425588298e-15
